In [3]:
import requests
import pandas

#links to the excel files containing the postal area and suburb
postal_area_link = "https://www.abs.gov.au/statistics/standards/australian-statistical-geography-standard-asgs-edition-3/jul2021-jun2026/access-and-downloads/allocation-files/POA_2021_AUST.xlsx"
suburb_link = "https://www.abs.gov.au/statistics/standards/australian-statistical-geography-standard-asgs-edition-3/jul2021-jun2026/access-and-downloads/allocation-files/SAL_2021_AUST.xlsx"

#download excel files and load them into a pandas dataframe
postal_area_df = pandas.read_excel(postal_area_link, engine='openpyxl')
suburb_df = pandas.read_excel(suburb_link, engine='openpyxl')



In [11]:
#join the two dataframes on MB_CODE_2021
merged_df = pandas.merge(postal_area_df, suburb_df, on='MB_CODE_2021')
merged_df = merged_df[['SAL_NAME_2021', 'POA_CODE_2021', 'STATE_NAME_2021']]

,SAL_NAME_2021,POA_CODE_2021,STATE_NAME_2021
0,Darwin City,0800,Northern Territory
1,Darwin City,0800,Northern Territory
2,Darwin City,0800,Northern Territory
3,Darwin City,0800,Northern Territory
4,Darwin City,0800,Northern Territory
...,...,...,...
368281,Migratory - Offshore - Shipping (Tas.),9797,Tasmania
368282,Migratory - Offshore - Shipping (Tas.),9797,Tasmania
368283,Migratory - Offshore - Shipping (NT),9797,Northern Territory
368284,Migratory - Offshore - Shipping (NT),9797,Northern Territory


In [16]:
#rename columns
merged_df = merged_df.rename(columns={'SAL_NAME_2021':'suburb', 'POA_CODE_2021':'postcode', 'STATE_NAME_2021':'state'})

#remove duplicates and rows with null values
merged_df = merged_df.drop_duplicates()
merged_df = merged_df.dropna()

#if multiple suburbs have the same postcode, keep the first one
merged_df = merged_df.drop_duplicates(subset='postcode', keep='first')

#remove rows with postcodes that are not numbers
merged_df = merged_df[merged_df['postcode'].str.isnumeric()]

#remove special postcodes
merged_df = merged_df[merged_df['postcode'] != '9494']
merged_df = merged_df[merged_df['postcode'] != '9797']

#map state to state code
state_code = {
    'Northern Territory':'NT',
    'Australian Capital Territory':'ACT',
    'New South Wales':'NSW',
    'Victoria':'VIC',
    'Queensland':'QLD',
    'South Australia':'SA',
    'Western Australia':'WA',
    'Tasmania':'TAS'
}
merged_df['state_code'] = merged_df['state'].map(state_code)

#save to csv
merged_df.to_csv('aus_postcode.csv', index=False)

#save to excel
merged_df.to_excel('aus_postcode.xlsx', index=False)


In [17]:
#select distinct states
states = merged_df['state'].unique()

In [22]:
# create a new column for state code
merged_df

,suburb,postcode,state,state_code
0,Darwin City,0800,Northern Territory,NT
1,Jingili,0810,Northern Territory,NT
2,Leanyer,0812,Northern Territory,NT
3,Stuart Park,0820,Northern Territory,NT
4,Rakula,0822,Northern Territory,NT
...,...,...,...,...
2638,Strahan,7468,Tasmania,TAS
2639,Granville Harbour,7469,Tasmania,TAS
2640,Rosebery (Tas.),7470,Tasmania,TAS
2641,No usual address (ACT),9494,Australian Capital Territory,ACT
